# Celltype Classification with Transcriptformer

This notebooks showcase how the Transcriptformer embeddings can be used to train a classifier for celltype classification.

In [ ]:
import json
import logging
import os

import hydra
from omegaconf import DictConfig, OmegaConf

from transcriptformer.model.inference import run_inference